In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
import matplotlib.pyplot as plt
import db_connection as db

In [9]:
model_path = os.path.join('models', 'transformer_tunned_model.keras')

# ---------- 1. Data Loading & Preprocessing ----------
# Ensure working directory contains 'flights_cleaned.csv'
df =  db.read_db("flights_cleaned")
df.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,36.980933,29.315317,2077,9.485190,34.146683,3.0,-12.0,3.0,3.161730,141,6.0,29.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
1,36.980883,29.315350,2077,6.292672,22.653618,3.0,-9.0,3.0,2.097557,151,10.0,30.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
2,36.980833,29.315383,2078,6.292672,22.653621,4.0,-5.0,4.0,1.573168,151,0.0,31.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
3,36.980767,29.315433,2080,8.634538,31.084336,3.0,-2.0,3.0,2.878179,149,2.0,32.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
4,36.980700,29.315483,2082,8.634540,31.084343,2.0,1.0,2.0,4.317270,149,0.0,33.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0


In [10]:
# Separate features and target
target_col = 'climb_rate_m/s'
feature_cols = [c for c in df.columns if c != target_col]
X = df[feature_cols].values.astype(np.float32)
y = df[target_col].values.astype(np.float32)

# Train-test split
split_frac = 0.8
split_idx = int(len(X) * split_frac)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# Save scaler
os.makedirs('models', exist_ok=True)
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [11]:

# ---------- 2. Create sliding-window datasets ----------
T = 10  # sequence length
batch_size = 32
train_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_train, y_train,
    sequence_length=T,
    sequence_stride=1,
    batch_size=batch_size,
    shuffle=False
)
test_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_test, y_test,
    sequence_length=T,
    sequence_stride=1,
    batch_size=batch_size,
    shuffle=False
)


In [12]:

# ---------- 3. Positional Encoding ----------
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, None]
    i   = np.arange(d_model)[None, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / d_model)
    angle_rads = pos * angle_rates
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    return tf.cast(np.concatenate([sines, cosines], axis=-1), tf.float32)


In [13]:

# ---------- 4. Build Transformer Model ----------
seq_len = T
n_feats = X.shape[1]
d_model = 64
num_heads = 4

def build_transformer(seq_len, n_feats, d_model, num_heads):
    inp = layers.Input(shape=(seq_len, n_feats))
    x = layers.Dense(d_model)(inp)
    pos_enc = get_positional_encoding(seq_len, d_model)
    x = x + pos_enc
    # Encoder block
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn)
    ffn = tf.keras.Sequential([
        layers.Dense(d_model*2, activation='relu'),
        layers.Dense(d_model)
    ])
    x = layers.LayerNormalization(epsilon=1e-6)(x + ffn(x))
    x = layers.GlobalAveragePooling1D()(x)
    out = layers.Dense(1)(x)
    return Model(inp, out, name='Transformer')

model = build_transformer(seq_len, n_feats, d_model, num_heads)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "Transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 10, 18)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 10, 64)    │      1,216 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 10, 64)    │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 64)    │     66,368 │ add_3[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 10, 64)    │          0 │ add_3[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 64)    │        128 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 10, 64)    │     16,576 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 10, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 64)    │        128 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1)         │         65 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 84,481 (330.00 KB)

 Trainable params: 84,481 (330.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# ---------- 5. Train ----------
callbacks = [
    EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True),
    ModelCheckpoint(
        filepath=model_path,
        monitor='val_mae',
        save_best_only=True
    )
]

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 154s 8ms/step - loss: 0.0760 - mae: 0.1056 - val_loss: 0.0036 - val_mae: 0.0425
Epoch 2/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 169s 9ms/step - loss: 0.0017 - mae: 0.0246 - val_loss: 0.0024 - val_mae: 0.0391
Epoch 3/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 171s 9ms/step - loss: 8.1232e-04 - mae: 0.0160 - val_loss: 8.0218e-04 - val_mae: 0.0176
Epoch 4/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 148s 8ms/step - loss: 0.0011 - mae: 0.0132 - val_loss: 4.5270e-04 - val_mae: 0.0169
Epoch 5/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 142s 8ms/step - loss: 4.1138e-04 - mae: 0.0107 - val_loss: 5.5727e-04 - val_mae: 0.0157
Epoch 6/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 146s 8ms/step - loss: 4.7168e-04 - mae: 0.0117 - val_loss: 4.9011e-04 - val_mae: 0.0115
Epoch 7/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 149s 8ms/step - loss: 3.9100e-04 - mae: 0.0096 - val_loss: 7.0800e-04 - val_mae: 0.0201
Epoch 8/15
18824/18824 ━━━━━━━━━━━━━━━━━━━━ 139s 7ms/step - loss: 2.6829e-04 - mae: 0.0082 - val

In [2]:

# ---------- 6. Save Model ----------
model.save(model_path)
print(f'Model saved to {model_path}')

# ---------- 7. Evaluate & Plot Errors ----------
y_pred = model.predict(test_ds).flatten()
y_true = np.concatenate([y for _, y in test_ds], axis=0)

mae = np.mean(np.abs(y_true - y_pred))
rmse = np.sqrt(np.mean((y_true - y_pred)**2))
print(f'Test MAE: {mae:.4f}, RMSE: {rmse:.4f}')

# Error histogram
errors = y_true - y_pred
plt.figure()
plt.hist(errors, bins=50)
plt.title('Transformer Prediction Errors')
plt.xlabel('Error (m/s)')
plt.ylabel('Count')
plt.show()

NameError: name 'model' is not defined